In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
movie= pd.read_csv(r'C:\Users\CL2797\Desktop\ML\Recommender sysytems\movies.csv', usecols=['movieId','title'], dtype={'movieId': 'int32', 'title':'str'}
                )
rating=pd.read_csv(r'C:\Users\CL2797\Desktop\ML\Recommender sysytems\ratings.csv', usecols =['userId', 'movieId', 'rating'], dtype={'userId':'int32', 'movieId':'int32', 'rating':'float32'})

In [3]:
movie.head()
rating.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [4]:
df=pd.merge(movie, rating,on='movieId')

In [5]:
df.head()

,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5


In [6]:
df.isnull().sum()

movieId    0
title      0
userId     0
rating     0
dtype: int64

In [7]:
df.describe()

,movieId,userId,rating
count,100836.000000,100836.000000,100836.000000
mean,19435.295718,326.127564,3.501557
std,35530.987199,182.618491,1.042575
min,1.000000,1.000000,0.500000
25%,1199.000000,177.000000,3.000000
50%,2991.000000,325.000000,3.500000
75%,8122.000000,477.000000,4.000000
max,193609.000000,610.000000,5.000000


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   movieId  100836 non-null  int32  
 1   title    100836 non-null  object 
 2   userId   100836 non-null  int32  
 3   rating   100836 non-null  float32
dtypes: float32(1), int32(2), object(1)
memory usage: 2.7+ MB


In [9]:
df.shape

(100836, 4)

In [10]:
df.head()

,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5


In [11]:
movie_ratingcount= df.groupby('title')['rating'].count().reset_index().rename(columns = {'rating': 'totalRatingCount'})

## if we take just count, it will be series, so we use reset index to convert it to dataframe. 

In [12]:
movie_ratingcount.head()

,title,totalRatingCount
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2


In [13]:
rating_with_totalRatingCount = df.merge(movie_ratingcount, left_on = 'title', right_on = 'title', how = 'left')
rating_with_totalRatingCount.head()

,movieId,title,userId,rating,totalRatingCount
0,1,Toy Story (1995),1,4.0,215
1,1,Toy Story (1995),5,4.0,215
2,1,Toy Story (1995),7,4.5,215
3,1,Toy Story (1995),15,2.5,215
4,1,Toy Story (1995),17,4.5,215


In [17]:
pd.set_option('display.float_format', lambda x: '%.1f' % x)
print(movie_ratingcount['totalRatingCount'].describe())

count   9719.0
mean      10.4
std       22.4
min        1.0
25%        1.0
50%        3.0
75%        9.0
max      329.0
Name: totalRatingCount, dtype: float64


In [23]:
popularity_threshold=50
popular_movie=rating_with_totalRatingCount.query('totalRatingCount>=@popularity_threshold')

In [24]:
popular_movie.head()

,movieId,title,userId,rating,totalRatingCount
0,1,Toy Story (1995),1,4.0,215
1,1,Toy Story (1995),5,4.0,215
2,1,Toy Story (1995),7,4.5,215
3,1,Toy Story (1995),15,2.5,215
4,1,Toy Story (1995),17,4.5,215


In [29]:
# creating a pivot table
movie_feature= popular_movie.pivot_table(index='title', columns='userId', values='rating').fillna(0)
# movie_features_df=rating_popular_movie.pivot_table(index='title',columns='userId',values='rating').fillna(0)


In [30]:
movie_feature.head()


userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men (1957),0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001: A Space Odyssey (1968),0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,5.0,0.0,3.0,0.0,4.5
28 Days Later (2002),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,5.0
300 (2007),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,4.0


In [32]:
from scipy.sparse import csr_matrix

movie_feature_matrix = csr_matrix(movie_feature.values)

from sklearn.neighbors import NearestNeighbors


model_NN = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_NN.fit(movie_feature_matrix)


NearestNeighbors(algorithm='brute', metric='cosine')

In [35]:
(movie_feature_matrix)

<450x606 sparse matrix of type '<class 'numpy.float32'>'
	with 41360 stored elements in Compressed Sparse Row format>

In [37]:
query_index = np.random.choice(movie_feature.shape[0]) #picking a random movie
print(query_index)
distances, indices = model_NN.kneighbors(movie_feature.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)
#finding 6 nearest neighbor to 342th movies. The fist movies (closest to 342th) will be the movie itself.


342


In [38]:
distances

array([[0.        , 0.44582587, 0.44909662, 0.4508266 , 0.45365626,
        0.45418674]], dtype=float32)

In [39]:
indices

array([[342, 445, 213, 308, 444, 357]], dtype=int64)

In [40]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(movie_feature.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, movie_feature.index[indices.flatten()[i]], distances.flatten()[i]))


Recommendations for School of Rock (2003):

1: X2: X-Men United (2003), with distance of 0.44582587480545044:
2: Incredibles, The (2004), with distance of 0.4490966200828552:
3: Pirates of the Caribbean: The Curse of the Black Pearl (2003), with distance of 0.45082658529281616:
4: X-Men: The Last Stand (2006), with distance of 0.45365625619888306:
5: Sin City (2005), with distance of 0.45418673753738403:
